# ADES
This notebook demonstrates deployment and execution of workflows on ADES.

Instantiate ADES client:

In [1]:
from requests import HTTPError
import os

import pyeodh

username = os.getenv("ADES_USER")
password = os.getenv("ADES_PASSWORD")

client = pyeodh.Client(auth=(username, password))
ades = client.get_ades()

Get available processes

In [2]:
for p in ades.get_processes():
    print(p.id)

display
echo
produce-indicator
convert-url


Get list of previously executed jobs

In [3]:
for j in ades.get_jobs():
    print(j.id, j.process_id, j.status)

5aad4340-1433-11ef-8ba3-5e783c18dea9 convert-url failed
470164c2-0c7f-11ef-bd64-fa92a27f4c9c produce-indicator running
ae91da94-1432-11ef-ba09-5e783c18dea9 convert-url successful
f59a291e-1432-11ef-912e-5e783c18dea9 convert-url failed
e8bff528-1439-11ef-bd9d-5e783c18dea9 convert-url successful


Undeploy a process so we can create again later

In [12]:
try:
    ades.get_process("convert-url").delete()
except HTTPError:
    print("Process not found, no need to undeploy.")

Deploy a process, referencing the CWL file by URL

In [13]:
convert_url_proc = ades.deploy_process(
    cwl_url="https://raw.githubusercontent.com/EOEPCA/deployment-guide/main/deploy/samples/requests/processing/convert-url-app.cwl"
)
print(convert_url_proc.id, convert_url_proc.description)

convert-url Convert URL


Or update a process by passing the CWL file content directly.
Both `Ades.deploy_process()` and `Process.update()` can handle either a URL or a CWL YAML.
Here we update the `convert-url` process with a slightly modified description.

In [6]:
cwl_yaml = """cwlVersion: v1.0
$namespaces:
  s: https://schema.org/
s:softwareVersion: 0.1.2
schemas:
  - http://schema.org/version/9.0/schemaorg-current-http.rdf
$graph:
  # Workflow entrypoint
  - class: Workflow
    id: convert-url
    label: convert url app
    doc: Convert URL YAML
    requirements:
      ResourceRequirement:
        coresMax: 1
        ramMax: 1024
    inputs:
      fn:
        label: the operation to perform
        doc: the operation to perform
        type: string
        default: "resize"
      url:
        label: the image to convert
        doc: the image to convert
        type: string
        default: "https://eoepca.org/media_portal/images/logo6_med.original.png"
      size:
        label: the percentage for a resize operation
        doc: the percentage for a resize operation
        type: string
        default: "50%"
    outputs:
      - id: converted_image
        type: Directory
        outputSource:
          - convert/results
    steps:
      convert:
        run: "#convert"
        in:
          fn: fn
          url: url
          size: size
        out:
          - results
  # convert.sh - takes input args `--url`
  - class: CommandLineTool
    id: convert
    requirements:
      ResourceRequirement:
        coresMax: 1
        ramMax: 512
    hints:
      DockerRequirement:
        dockerPull: eoepca/convert:latest
    baseCommand: convert.sh
    inputs:
      fn:
        type: string
        inputBinding:
          position: 1
      url:
        type: string
        inputBinding:
          position: 2
          prefix: --url
      size:
        type: string
        inputBinding:
          position: 3
    outputs:
      results:
        type: Directory
        outputBinding:
          glob: .
"""

convert_url_proc.update(cwl_yaml=cwl_yaml)
print(convert_url_proc.id, convert_url_proc.description)

convert-url Convert URL YAML


Let's execute our deployed process

In [14]:
convert_url_job = convert_url_proc.execute(
    {
        "inputs": {
            "fn": "resize",
            "url": "https://eoepca.org/media_portal/images/logo6_med.original.png",
            "size": "50%",
        }
    }
)

print(convert_url_job.id, convert_url_job.status, convert_url_job.message)

763abe6e-143b-11ef-a71e-5e783c18dea9 running ZOO-Kernel accepted to run your service!


... and refresh the job status

In [16]:
convert_url_job.refresh()
print(convert_url_job.id, convert_url_job.status, convert_url_job.message)

763abe6e-143b-11ef-a71e-5e783c18dea9 running workflow wrapped, creating processing environment


or continually poll

In [19]:
from pyeodh.ades import AdesJobStatus
import time


old_status = ""
old_message = ""
while convert_url_job.status == AdesJobStatus.RUNNING:
    time.sleep(2)
    convert_url_job.refresh()
    if convert_url_job.status != old_status:
        print("\n")
        print(f"Status: {convert_url_job.status}")
    if convert_url_job.message != old_message:
        print(f"Message: {convert_url_job.message}")

    old_status = convert_url_job.status
    old_message = convert_url_job.message